In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}


In [3]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    data = []
    state = None
    state_towns = []
    with open('university_towns.txt') as file:
        for line in file:
            thisLine = line[:-1]
            if thisLine[-6:] == '[edit]': # State durch edit am Ende finden
                state = thisLine[:-6]
                continue
            if '(' in line:
                town = thisLine[:thisLine.index('(')-1] # Stadt steht vor der Klammer
                state_towns.append([state,town])
            else:
                town = thisLine # Stadt 
                state_towns.append([state,town])
            data.append(thisLine)
    df = pd.DataFrame(state_towns,columns = ['State','RegionName']) # Spalten
    return df
get_list_of_university_towns()

State                    RegionName
0          Alabama                        Auburn
1          Alabama                      Florence
2          Alabama                  Jacksonville
3          Alabama                    Livingston
4          Alabama                    Montevallo
5          Alabama                          Troy
6          Alabama                    Tuscaloosa
7          Alabama                      Tuskegee
8           Alaska                     Fairbanks
9          Arizona                     Flagstaff
10         Arizona                         Tempe
11         Arizona                        Tucson
12        Arkansas                   Arkadelphia
13        Arkansas                        Conway
14        Arkansas                  Fayetteville
15        Arkansas                     Jonesboro
16        Arkansas                      Magnolia
17        Arkansas                    Monticello
18        Arkansas                  Russellville
19        Arkansas                        Searcy
20      California                        Angwin
21      California                        Arcata
22      California                      Berkeley
23      California                         Chico
24      California                     Claremont
25      California                        Cotati
26      California                         Davis
27      California                        Irvine
28      California                    Isla Vista
29      California  University Park, Los Angeles
..             ...                           ...
487       Virginia                          Wise
488       Virginia                    Chesapeake
489     Washington                    Bellingham
490     Washington                        Cheney
491     Washington                    Ellensburg
492     Washington                       Pullman
493     Washington  University District, Seattle
494  West Virginia                        Athens
495  West Virginia                    Buckhannon
496  West Virginia                      Fairmont
497  West Virginia                     Glenville
498  West Virginia                    Huntington
499  West Virginia                    Montgomery
500  West Virginia                    Morgantown
501  West Virginia                 Shepherdstown
502  West Virginia                  West Liberty
503      Wisconsin                      Appleton
504      Wisconsin                    Eau Claire
505      Wisconsin                     Green Bay
506      Wisconsin                     La Crosse
507      Wisconsin                       Madison
508      Wisconsin                     Menomonie
509      Wisconsin                     Milwaukee
510      Wisconsin                       Oshkosh
511      Wisconsin                   Platteville
512      Wisconsin                   River Falls
513      Wisconsin                 Stevens Point
514      Wisconsin                      Waukesha
515      Wisconsin                    Whitewater
516        Wyoming                       Laramie

[517 rows x 2 columns]

In [4]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    df_gdp = pd.read_excel('gdplev.xls', skiprows=220, parse_cols='E,G', header=None) # Einlesen und parsen gdplev.xls
    df_gdp.columns = ['quarter', 'GDP'] # Spalten Viertel und GDP

    for i in range(4, len(df_gdp)): #A recession period is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth. 
        if (df_gdp.loc[i - 4, 'GDP'] > df_gdp.loc[i - 3, 'GDP']) \
                and (df_gdp.loc[i - 3, 'GDP'] > df_gdp.loc[i - 2, 'GDP']) \
                and (df_gdp.loc[i - 2, 'GDP'] < df_gdp.loc[i - 1, 'GDP']) \
                and (df_gdp.loc[i - 1, 'GDP'] < df_gdp.loc[i, 'GDP']):
            recession_base_idx = i - 4 # Index recession

    result = df_gdp.loc[recession_base_idx, 'quarter']
    j = recession_base_idx
    while True:
        if df_gdp.loc[j - 1, 'GDP'] > df_gdp.loc[j, 'GDP']:
            j -= 1
        else:
            result = df_gdp.loc[j + 1, 'quarter'] # recession start gefunden
            break

    return result
get_recession_start()

'2008Q3'

In [5]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
       
    df_gdp = pd.read_excel('gdplev.xls', skiprows=220, parse_cols='E,G', header=None)
    df_gdp.columns = ['quarter', 'GDP']

    for i in range(4, len(df_gdp)):
        if (df_gdp.loc[i-4, 'GDP'] > df_gdp.loc[i-3, 'GDP']) \
                and (df_gdp.loc[i-3, 'GDP'] > df_gdp.loc[i-2, 'GDP']) \
                and (df_gdp.loc[i-2, 'GDP'] < df_gdp.loc[i-1, 'GDP']) \
                and (df_gdp.loc[i-1, 'GDP'] < df_gdp.loc[i, 'GDP']):
            result = df_gdp.loc[i, 'quarter'] #recession end gefunden

    return result


get_recession_end()

'2009Q4'

In [6]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    df_gdp = pd.read_excel('gdplev.xls', skiprows=220, parse_cols='E,G', header=None)
    df_gdp.columns = ['quarter', 'GDP']

    for i in range(4, len(df_gdp)):
        if (df_gdp.loc[i-4, 'GDP'] > df_gdp.loc[i-3, 'GDP']) \
                and (df_gdp.loc[i-3, 'GDP'] > df_gdp.loc[i-2, 'GDP']) \
                and (df_gdp.loc[i-2, 'GDP'] < df_gdp.loc[i-1, 'GDP']) \
                and (df_gdp.loc[i-1, 'GDP'] < df_gdp.loc[i, 'GDP']):
            result = df_gdp.loc[i-2, 'quarter'] # Tiefpunkt recession gefunden

    return result

get_recession_bottom()

'2009Q2'

In [7]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    df = pd.read_csv('City_Zhvi_AllHomes.csv', header=0)

    cols_to_keep = ['RegionID', 'RegionName', 'State'] # Erstellen der Spalten
    for i in range(2000, 2017):
        for j in range(1, 13):
            if j <= 9:
                if i == 2016 and j == 9:
                    pass
                else:
                    month_str = '0' + str(j)
            else:
                if i == 2016:
                    pass
                else:
                    month_str = str(j)
            cols_to_keep.append(str(i) + '-' + month_str)
    df = df[cols_to_keep]

    df['State'] = df['State'].replace(states)

    def convert_to_quarter(ym): # Einteilung der Monate in Viertel
        year, month = ym.split('-')
        if month == '01' or month == '02' or month == '03':
            result = year + 'q1'
        elif month == '04' or month == '05' or month == '06':
            result = year + 'q2'
        elif month == '07' or month == '08' or month == '09':
            result = year + 'q3'
        else:
            result = year + 'q4'
        return result

    # Erstellen des Dataframe & Säubern
    df_compiled = df.set_index(['State', 'RegionName', 'RegionID']).stack(dropna=False)
    df_compiled = df_compiled.reset_index().rename(columns={'level_3': 'year_month', 0: 'gdp'})
    df_compiled.drop_duplicates(inplace=True)
    df_compiled['quarter'] = df_compiled['year_month'].apply(convert_to_quarter)
    df_compiled = df_compiled.drop('year_month', axis=1)
    result = df_compiled.pivot_table(values='gdp', index=['State', 'RegionName', 'RegionID'], columns='quarter', aggfunc=np.mean)
    result = result.reset_index()
    result = result.drop('RegionID', axis=1)

    result = result.set_index(['State', 'RegionName'])
    return result

convert_housing_data_to_quarters()

quarter                                     2000q1         2000q2  \
State     RegionName                                                
Alabama   Adamsville                  69033.333333   69166.666667   
          Alabaster                  122133.333333  123066.666667   
          Alexander City                       NaN            NaN   
          Anderson                    69933.333333   73166.666667   
          Argo                       136966.666667  134666.666667   
          Ashford                     88566.666667   92666.666667   
          Ashville                   105300.000000  102433.333333   
          Attalla                     41866.666667   43733.333333   
          Auburn                     157266.666667  159066.666667   
          Autaugaville                         NaN            NaN   
          Axis                                 NaN            NaN   
          Baileyton                   64166.666667   63200.000000   
          Bay Minette                 81700.000000   78533.333333   
          Bayou La Batre              44066.666667   44500.000000   
          Bessemer                             NaN            NaN   
          Billingsley                          NaN            NaN   
          Birmingham                  54033.333333   54400.000000   
          Blountsville                80900.000000   78033.333333   
          Brighton                             NaN            NaN   
          Brookwood                   92566.666667   95100.000000   
          Buhl                        90800.000000   94600.000000   
          Calera                     108933.333333  110366.666667   
          Center Point                80966.666667   81200.000000   
          Chalkville                  94100.000000   94433.333333   
          Chelsea                    162066.666667  167033.333333   
          Cherokee                             NaN            NaN   
          Chickasaw                   51200.000000   53666.666667   
          Chunchula                   80266.666667   81766.666667   
          Citronelle                  64833.333333   66633.333333   
          Clay                       120900.000000  122266.666667   
...                                            ...            ...   
Wisconsin Woodville                            NaN            NaN   
          Yorkville                  139066.666667  134900.000000   
Wyoming   Antelope Valley-Crestview  222400.000000  222400.000000   
          Bar Nunn                    93033.333333   89666.666667   
          Basin                                NaN            NaN   
          Buffalo                    269500.000000  269500.000000   
          Burns                      101533.333333  104566.666667   
          Casper                      89233.333333   89600.000000   
          Cheyenne                   116866.666667  120033.333333   
          Cody                                 NaN            NaN   
          Douglas                    190900.000000  190900.000000   
          Dubois                               NaN            NaN   
          Evanston                   154400.000000  154400.000000   
          Evansville                 128033.333333  128766.666667   
          Gillette                   199500.000000  199500.000000   
          Jackson                              NaN            NaN   
          Kemmerer                   169900.000000  169933.333333   
          Lander                     230000.000000  230000.000000   
          Laramie                              NaN            NaN   
          Lovell                     159466.666667  159500.000000   
          Lyman                      187500.000000  187500.000000   
          Newcastle                            NaN            NaN   
          Pine Bluffs                 93733.333333   95066.666667   
          Rawlins                              NaN            NaN   
          Riverton                   187500.000000  187500.000000  

In [8]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    data = convert_housing_data_to_quarters().copy()
    data = data.loc[:,'2008q3':'2009q2']
    data = data.reset_index()
    
    #decline or growth of housing prices
    def price_ratio(row):
        return (row['2008q3'] - row['2009q2'])/row['2008q3']
    
    data['up&down'] = data.apply(price_ratio,axis=1) 
    
    uni_town = get_list_of_university_towns()['RegionName']
    uni_town = set(uni_town)

    # Einteilung 1 oder 0 für ist uni_town oder nicht 
    def is_uni_town(row):
        if row['RegionName'] in uni_town:
            return 1
        else:
            return 0
    data['is_uni'] = data.apply(is_uni_town,axis=1)
    
    not_uni = data[data['is_uni']==0].loc[:,'up&down'].dropna()
    is_uni  = data[data['is_uni']==1].loc[:,'up&down'].dropna()
    
    #t-test
    def better():
        if not_uni.mean() < is_uni.mean():
            return 'non-university town'
        else:
            return 'university town'
    p_val = list(ttest_ind(not_uni, is_uni))[1]
    result = (True,p_val,better())
    return result

run_ttest()

(True, 0.00029249735039167657, 'university town')